In [ ]:

#  Install Libraries (Colab)

!pip install transformers datasets scikit-learn arabert torch -q


#  Import Libraries

import pandas as pd
import numpy as np
import torch
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
from arabert.preprocess import ArabertPreprocessor


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 2.9 MB/s eta 0:00:00


In [ ]:
# 3️⃣ Load Dataset
df = pd.read_excel("story_review_v2.xlsx")

print(df.head())
print(df["sentiment"].value_counts())

                                          comment sentiment        aspect
0       التجربة ما عجبتني خصوصًا في موضوع السعر 💔      سلبي         السعر
1        التجربة عادية خصوصًا في موضوع جودة صراحة     محايد          جودة
2         للأسف ما كنت راضي خصوصًا في موضوع السعر      سلبي         السعر
3                       تجربه سيئه نوعًا ما صراحة      سلبي          جودة
4  النتيجة خيبت أملي خصوصًا في موضوع سرعة التنفيذ      سلبي  سرعة التنفيذ
sentiment
سلبي      1200
إيجابي    1200
محايد      800
Name: count, dtype: int64


In [ ]:



#  Clean Arabic Text
def clean_text(text):
    text = str(text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"[^\u0600-\u06FF\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["comment"] = df["comment"].apply(clean_text)

# 5️ Label Encoding
le = LabelEncoder()
df["label"] = le.fit_transform(df["sentiment"])

label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("Label Mapping:", label_mapping)

# 6️ Train / Val / Test Split
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    df["comment"], df["label"],
    test_size=0.2,
    stratify=df["label"],
    random_state=42
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels,
    test_size=0.5,
    stratify=temp_labels,
    random_state=42
)

Label Mapping: {'إيجابي': np.int64(0), 'سلبي': np.int64(1), 'محايد': np.int64(2)}


In [ ]:


# 7️ Load AraBERT
model_name = "aubmindlab/bert-base-arabertv02"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3
)

arabert_prep = ArabertPreprocessor(model_name=model_name)

train_texts = train_texts.apply(arabert_prep.preprocess)
val_texts = val_texts.apply(arabert_prep.preprocess)
test_texts = test_texts.apply(arabert_prep.preprocess)

# 8️ Tokenization
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=128)

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = list(labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_encodings, train_labels)
val_dataset = Dataset(val_encodings, val_labels)
test_dataset = Dataset(test_encodings, test_labels)

# 9️ Compute Metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

#  Training Arguments

training_args = TrainingArguments(
    output_dir="./arabert-results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    save_total_limit=2,
    warmup_ratio=0.1,
    fp16=True
)

#  Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

#  Train Model
trainer.train()

#  Evaluate on Test Set
results = trainer.evaluate(test_dataset)
print("Test Results:", results)

#  Confusion Matrix
predictions = trainer.predict(test_dataset)
y_true = test_labels
y_pred = np.argmax(predictions.predictions, axis=1)

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=le.classes_))

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

#  Save Model
trainer.save_model("./final_arabert_model")
tokenizer.save_pretrained("./final_arabert_model")


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: aubmindlab/bert-base-arabertv02
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
bert.embeddings.position_ids               | UNEXPECTED | 
classifier.bias                            | MISSING    | 
classifier.weight                          | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Conside

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.001014,1.000000,1.000000,1.000000,1.000000
2,No log,0.000332,1.000000,1.000000,1.000000,1.000000
3,No log,0.000216,1.000000,1.000000,1.000000,1.000000


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


...

Test Results: {'eval_loss': 0.0010092331795021892, 'eval_accuracy': 1.0, 'eval_f1': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_runtime': 22.7715, 'eval_samples_per_second': 14.053, 'eval_steps_per_second': 0.878, 'epoch': 3.0}


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Classification Report:
              precision    recall  f1-score   support

      إيجابي       1.00      1.00      1.00       120
        سلبي       1.00      1.00      1.00       120
       محايد       1.00      1.00      1.00        80

    accuracy                           1.00       320
   macro avg       1.00      1.00      1.00       320
weighted avg       1.00      1.00      1.00       320

Confusion Matrix:
[[120   0   0]
 [  0 120   0]
 [  0   0  80]]


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

('./final_arabert_model/tokenizer_config.json',
 './final_arabert_model/tokenizer.json')

In [ ]:
from transformers import pipeline

pipe = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

test_sentences = [
    "المنتج رائع جداً والجودة ممتازة",
    "التوصيل تأخر كثيراً ولم يكن التغليف جيداً",
    "السعر مرتفع قليلاً لكن الخدمة جيدة"
]

for sentence in test_sentences:
    preprocessed_text = arabert_prep.preprocess(sentence)
    result = pipe(preprocessed_text)
    print(f"النص: {sentence} \n التصنيف: {result}\n")

النص: المنتج رائع جداً والجودة ممتازة 
 التصنيف: [{'label': 'LABEL_0', 'score': 0.997723400592804}]

النص: التوصيل تأخر كثيراً ولم يكن التغليف جيداً 
 التصنيف: [{'label': 'LABEL_1', 'score': 0.9985729455947876}]

النص: السعر مرتفع قليلاً لكن الخدمة جيدة 
 التصنيف: [{'label': 'LABEL_2', 'score': 0.5770113468170166}]



In [ ]:
model.save_pretrained("./my_final_model")
tokenizer.save_pretrained("./my_final_model")

import shutil
shutil.make_archive("my_model_archive", 'zip', "./my_final_model")

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

'/content/my_model_archive.zip'